# BioProjectメタデータを未病DBに転記する

このノートブックは、合原ムーンショットプロジェクトで開発された包括的未病データベース（未病DB）にメタデータを登録する際の便宜を図るものです。

* 対象ユーザー: 未病DBにプロジェクトメタデータを登録する方
* 入力元: BioProject形式のメタデータが記載されたExcelファイル（DDBJセンターが作成したフォーマット）
* 出力先: JSON形式の文字列（クリップボード経由で未病DBに貼り付ける）

## 1. 設定

入力元となるExcelファイルをこのノートブックと同じフォルダに置き、ファイル名を下のセルに設定してください。

In [ ]:
# 入力元Excelファイル名
input_file = 'BioProject_test1.xlsx'

必要に応じて、項目名の対応表を修正してください。

* BioProject項目名は、Excelフォーマットのシート名、Category名、Item名をピリオドで繋げて書きます。ただし、大文字は小文字に、空白は下線 `_` に置き換えます。例えば、シート="GENERAL INFO" の Category="Project Description" の Item="Project title" は `general_info.project_description.project_title` と書きます。
* Excelに同一のCategoryが複数ある場合は、次のように書きます。
  * `submitter.contact_person.0.e-mail` → 一番上の e-mail だけを出力します
  * `submitter.contact_person.1.e-mail` → 上から2番目の e-mail だけを出力します
  * `submitter.contact_person.:.e-mail` → すべての e-mail を結合して出力します
* 複数のItemをひとつの未病DB項目に対応させるには、コロン `:` を用いて以下のように書きます。
  * `submitter.contact_person.:.name` → first_name と last_name を合わせて1項目とします
  * `general_info.grants.:.*` → agency, agency_abbreviation, grant_id, grant_title を合わせて1項目とします
  * `publication.publication.:.*` → PubMed ID と DOI を合わせて1項目とします

In [ ]:
# '未病DB項目名':'BioProject項目名'
mapping = {
  'title-of-dataset':'',
  'title-of-dataset-en':'',
  'project-name':'',
  'data-id':'',
  'date-registered-in-metadata':'',
  'date-updated-in-metadata':'',
  'purpose-of-experiment':'general_info.project_description.project_title',
  'purpose-of-experiment-en':'',
  'description-of-experimental-condition':'general_info.project_description.description',
  'description-of-experimental-condition-en':'',
  'keyword-arr':'',
  'keyword-arr-en':'',
  'dataset-research-field':'',
  'Analysis-type':[],
  'Analysis-type-other':'project_type.project_data_type.project_data_type_description',
  'the-presence-of-metadata-files-created-for-a-specific-modality-in-other-databases':'',
  'metadata-filename':'',
  'necessity-of-contact-and-permission':'',
  'necessity-of-including-in-acknowledgments':'',
  'names-to-be-included-in-the-acknowledgments':'general_info.grants.:.*',
  'names-to-be-included-in-the-acknowledgments-en':'',
  'other-conditions-or-special-notes':'',
  'other-conditions-or-special-notes-en':'',
  'data-policy-license':'',
  'data-policy-free':'',
  'availability-of-commercial-use':'',
  'access-rights':'',
  'scheduled-release-date':'',
  'repository-information':'',
  'repository-url-doi-link':'',
  'other-supplementary-information':'publication.publication.:.*',
  'other-supplementary-information-en':'',
  'name-of-data-creator':'',
  'name-of-data-creator-en':'',
  'belonging-of-data-creator':'',
  'belonging-of-data-creator-en':'',
  'contact-of-data-creator':'',
  'name-of-data-manager':'submitter.contact_person.:.name',
  'name-of-data-manager-en':'',
  'belonging-of-data-manager':'submitter.organization.submitting_organization',
  'belonging-of-data-manager-en':'',
  'contact-of-data-manager':'submitter.contact_person.:.e-mail',
  'target-type-of-acquired-data':'target.organism_information.organism_name',
  'target-type-of-acquired-data-en':'',
  'ethics-review-committee-approval':'',
  'ethics-review-committee-approval-en':'',
  'informed-consent':'',
  'consent-for-provision-to-a-third-party':'',
  'overseas-offerings':'',
  'industrial-use':'',
  'ic-is-no':'',
  'anonymous-processing':'',
  'conflict-of-interest-Yes-or-No':'',
  'conflict-of-interest':'',
  'conflict-of-interest-en':'',
  'remarks-3':'',
  'remarks-3-en':''
}

複数のItemをひとつの未病DB項目に対応させる場合の区切り文字を設定してください。

In [ ]:
separator = ','

## 2. 実行

以下のセルをすべて実行してください。

In [ ]:
import pandas as pd
import json
import re
import warnings
from dotty_dict import dotty

# DDBJセンターが作成したExcelフォーマットを読み込む
def read_bioproject_from_excel(excel_file):
  # 読み込むワークシート名（原文ママ）
  sheets_to_read = ['SUBMITTER', 'GENERAL INFO', 'TARGET', 'PROJECT TYPE', 'PUBLICATION']
  # 複数のアイテムを含むカテゴリ名（小文字、空白→下線）
  plural_categories = ['contact_person', 'external_links', 'grants', 'publication']
  data = {}
  warnings.simplefilter(action='ignore', category=UserWarning)
  dfs = pd.read_excel(excel_file, sheet_name=sheets_to_read, header=1)
  for sheet_name in dfs:
    df = dfs[sheet_name]
    k = 0 # index of plural category
    if 'Category' in df.columns:
      df['Category'] = df['Category'].ffill()
    if 'Item' in df.columns:
      df['Item'] = df['Item'].ffill()
    sheet_data = {}
    for index, row in df.iterrows():
      category = normalize(row['Category'])
      item = normalize(row['Item'])
      value = row['Value']
      if category not in plural_categories:
        if category not in sheet_data:
          sheet_data[category] = {}
        sheet_data[category][item] = value
      else:
        if category not in sheet_data:
          sheet_data[category] = [{}]
          k = 0
        if item in sheet_data[category][k]:
          sheet_data[category].append({})
          k += 1
        sheet_data[category][k][item] = interpret(value)
    data[normalize(sheet_name)] = sheet_data
  return data

# 項目名を正規化
def normalize(name):
  name = str(name).strip().replace(' ', '_').lower()
  name = re.sub(r'[　０-９]', '', name)
  return name

# 入力値を正規化
def interpret(value):
  # if isinstance(value, str):
  #   value = str(value).strip()
  value = str(value).strip()
  return value    

# 出力値を正規化
def represent(value):
  if value:
    value = str(value).strip()
  else:
    value = ''
  return value

In [ ]:
input_data = read_bioproject_from_excel(input_file)
# input_data

In [ ]:
dotty_data = dotty(input_data)
# dotty_data

必要に応じて、以下の書式を修正してください。

In [ ]:
a = dotty_data.get('submitter.contact_person')
for x in a:
  x['name'] = f"{x['first_name']} {x['last_name']}"

In [ ]:
a = dotty_data.get('general_info.grants')
for x in a:
  x['*'] = f"{x['agency']} ({x['agency_abbreviation']}) #{x['grant_id']} \"{x['grant_title']}\""

In [ ]:
a = dotty_data.get('publication.publication')
for x in a:
  s = []
  if x['pubmed_id']:
    s.append(f"PMID:{x['pubmed_id']}")
  if x['doi']:
    s.append(f"DOI:{x['doi']}")
  x['*'] = ' '.join(s)

In [ ]:
# 出力データを構成する
output_data = {}
for k, v in mapping.items():
  if v:
    x = dotty_data.get(v)
    if isinstance(x, list):
      output_data[k] = separator.join(x)
    else:
      output_data[k] = represent(x)
output_data

## 3. 結果取得

表示された文字列をクリッボードにコピーし、未病DBのプロジェクトメタデータ入力画面に貼り付けてください。

In [ ]:
print(json.dumps(output_data, indent=2))